In [138]:
%env CUDA_VISIBLE_DEVICES=GPU-8868e167-e666-53c7-6c41-d8e83081f07e

env: CUDA_VISIBLE_DEVICES=GPU-8868e167-e666-53c7-6c41-d8e83081f07e


In [1]:
import os
import pandas as pd

In [2]:
#split dataset with K-Fold cross validation

import os
import yaml
from pathlib import Path

images_path = "/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/wells_model/images"
labels_path = "/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/wells_model/labels"

#sort image and labels folder
image_files = [f for f in os.listdir(images_path) if f.endswith(".png")]
image_files.sort()

labels = list(Path(labels_path).glob("*.txt"))
labels.sort(key=lambda x: x.stem)  # sort by stem (filename without extension)

image_files

['20240823_161022.png',
 '20240823_161106.png',
 '20240823_161147.png',
 '20240828_104734.png',
 '20240829_133137.png',
 '20240830_103104.png',
 '20240902_121458.png',
 '20240907_042114.png',
 '20240910_151535.png',
 '20240912_113530.png',
 '20240912_134010.png',
 '20240912_134051.png',
 '20240913_161403.png',
 '20240917_150020.png',
 '20241005_060027.png',
 '20241007_131206.png',
 '20241008_140227.png',
 '20241008_140309.png',
 '20241009_100529.png',
 '20241009_104723.png',
 '20241009_104804.png',
 '20241010_114556.png',
 '20241012_030656.png',
 '20241012_164211.png',
 '20241013_131355.png',
 '20241015_095232.png',
 '20241021_215718.png',
 '20241022_115051.png',
 '20241023_094322.png',
 '20241104_111230.png',
 '20241104_111316.png',
 '20241104_111703.png',
 '20241104_111918.png',
 '20241104_112003.png',
 '20241104_134105.png',
 '20241104_134235.png',
 '20241104_134319.png',
 '20241105_102608.png',
 '20241123_130159.png',
 '20250117_103830.png',
 '20250117_103915.png',
 '20250117_16585

In [3]:
#read contents of YAML
yaml_file = "/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/wells_model/dataset.yaml"
with open(yaml_file, encoding='utf8') as y:
    data =yaml.safe_load(y)
classes = data['names']
cls_idx = list(range(len(classes)))
print('classes:', classes, '\nindices', cls_idx)

classes: ['Well', 'Label'] 
indices [0, 1]


In [4]:
index = [label.stem for label in labels] #remove .txt extension
labels_df = pd.DataFrame([], columns=cls_idx, index=index)
labels_df = labels_df.fillna(0.0) #replace na values with 0
labels_df.head()

/tmp/ipykernel_1714866/2525601978.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels_df = labels_df.fillna(0.0) #replace na values with 0


,0,1
20240823_161022,0.0,0.0
20240823_161106,0.0,0.0
20240823_161147,0.0,0.0
20240828_104734,0.0,0.0
20240829_133137,0.0,0.0


In [5]:
import random
from sklearn.model_selection import KFold

random.seed(0)
ksplit=5
kf = kf = KFold(n_splits=ksplit, shuffle=True, random_state=42)
kfolds=list(kf.split(labels_df))

folds = [f"split_{n}" for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=index, columns=folds)

for i, (train, val) in enumerate(kfolds, start=1):
    folds_df[f"split_{i}"].loc[labels_df.iloc[train].index] = "train"
    folds_df[f"split_{i}"].loc[labels_df.iloc[val].index] = "val"

folds_df

/tmp/ipykernel_1714866/3920743572.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  folds_df[f"split_{i}"].loc[labels_df.iloc[train].index] = "train"
/tmp/ipykernel_1714866/3920743572.py:14: FutureWarning: ChainedAssignmentError: behaviou

,split_1,split_2,split_3,split_4,split_5
20240823_161022,train,train,val,train,train
20240823_161106,train,train,train,val,train
20240823_161147,train,train,train,val,train
20240828_104734,val,train,train,train,train
20240829_133137,train,val,train,train,train
20240830_103104,val,train,train,train,train
20240902_121458,train,val,train,train,train
20240907_042114,train,train,train,train,val
20240910_151535,train,val,train,train,train
20240912_113530,train,train,val,train,train


In [6]:
import datetime
from pathlib import Path

images_path = Path("/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/wells_model/images")
kfold_path = Path("/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/wells_model")

images = []
images = list(images_path.glob("*png"))

# Create the necessary directories and dataset YAML files
save_path = Path(kfold_path / f"{ksplit}-Fold_Cross-val")
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "labels").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "labels").mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f"{split}_dataset.yaml"
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, "w") as ds_y:
        yaml.safe_dump(
            {
                "path": split_dir.as_posix(),
                "train": "train",
                "val": "val",
                "names": classes,
            },
            ds_y,
        )

print("kfold cv splitting complete!")

kfold cv splitting complete!


In [7]:
# Lastly, copy images and labels into the respective directory ('train' or 'val') for each split.
# DONT RE-RUN this block unless you want to re-copy files!

import shutil
from tqdm import tqdm

# Create a dictionary for faster lookup of labels
label_map = {label.stem: label for label in labels}

for img_file in tqdm(image_files, desc="Copying files"):
    # Find the corresponding image and label files using the stem
    stem = os.path.splitext(img_file)[0] #e.g. filename type
    label = label_map.get(stem)
    
    # Check if a label exists for the current image
    if label is None:
        print(f"Warning: No label found for image {img_file}. Skipping.")
        continue

    image_path = Path(images_path) / img_file

    for split, k_split in folds_df.loc[stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / "images"
        lbl_to_path = save_path / split / k_split / "labels"

        # Copy image and label files to new directory
        shutil.copy(image_path, img_to_path / img_file)
        shutil.copy(label, lbl_to_path / label.name)

print("File copying to kfolds completed.")

# The training loop using YOLO will follow this block.

Copying files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [00:43<00:00,  1.21it/s]

File copying to kfolds completed.


In [ ]:
#Train YOLO using K-Fold Data Splits
from ultralytics import YOLO
model_path = '/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/yolov8m.pt'

results = {}
metrics_list =[]

for k, dataset_yaml in enumerate(ds_yamls):
    model = YOLO(model_path) # model absolute path used within loop for brand new model weights for each fold
    results[k] = model.train(
        data = dataset_yaml,
        epochs=10000,  # Number of training epochs
        patience = 100,
        batch=4,
        #close_mosaic = 10,
        imgsz=1024,  # Image size
        lr0 = 0.0001,
        kobj = 1,
        cls = 50,
        save=True,  # Save the model after training
        project = r"/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/wells_model/Results",  # Save results in this directory
        name=f'fold_{k + 1}',  
        device = [0]
        )
        
    # Validate the trained model for this fold
    metrics = model.val(data=dataset_yaml)

    metrics_list.append({
    "fold": k+1,
    "mAP50-95": metrics.box.map,
    "mAP50": metrics.box.map50,
    "mAP75": metrics.box.map75,
    "precision": metrics.box.mp,
    "recall": metrics.box.mr
})

# Convert to DataFrame for easy averaging
df = pd.DataFrame(metrics_list)
print("\nPer-fold results:\n", df)

# Compute mean across folds
avg_results = df.mean(numeric_only=True)
print("\nAverage across folds:\n", avg_results)

print("Training completed!")

Ultralytics 8.3.214 🚀 Python-3.13.5 torch-2.8.0+cu128 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=50, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/wells_model/5-Fold_Cross-val/split_1/split_1_dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10000, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fold_

In [154]:
import os
import pandas as pd

path = "/home/lero/idrive/cmac/DDMAP/Image_analysis/Results"

df_path_per_fold = os.path.join(path, 'per_fold_results.csv')
df_path_Average = os.path.join(path, 'Average_results.csv')

df.to_csv(df_path_per_fold, index=False)
avg_results.to_csv(df_path_Average, index=False)


